In [2]:
#For testing that environment can run model standalone
from summarize_from_feedback import sample
from summarize_from_feedback.utils import experiment_helpers as utils
from summarize_from_feedback.query_response_model import ModelSpec
from summarize_from_feedback.tasks import TaskHParams, TaskQueryHParams, TaskResponseHParams
import torch
import os

os.environ["JOB_NAME"] = "local" #For MPI-dependent logging in sample.main

HParams = sample.HParams(
     model_spec = ModelSpec(
         load_path="https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4",
         short_name="sup4_ppo_rm4",
     ),
     task = TaskHParams(
         query = TaskQueryHParams(
             format_str = "SUBREDDIT: r/{subreddit}\n\nTITLE: {title}\n\nPOST: {post}\n\nTL;DR:",
             dataset = "tldr_3_filtered",
             length =  512,
             truncate_text = "\n",
             truncate_field = "post",
             pad_side = "left"   
         ),
         response = TaskResponseHParams(
             length = 48,
             truncate_token = 50256,  # endoftext
             ref_format_str = " {reference}",  # add a leading space 
         )
     ),
     query_dataset_split= "valid",
     num_queries = 1
    )

#print(utils.sup4_ppo_rm4())
sample.main(H = HParams)

copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4/done-sentinel /tmp/bf-dir-cache/az/openaipublic/summarize-from-feedback/models/sup4_ppo_rm4/done-sentinel
copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4/info.json /tmp/bf-dir-cache/az/openaipublic/summarize-from-feedback/models/sup4_ppo_rm4/info.json
copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4/checkpoint/final_layer_norm_shard_000.pkl /tmp/bf-dir-cache/az/openaipublic/summarize-from-feedback/models/sup4_ppo_rm4/checkpoint/final_layer_norm_shard_000.pkl
copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4/checkpoint/input_embeddings_shard_000.pkl /tmp/bf-dir-cache/az/openaipublic/summarize-from-feedback/models/sup4_ppo_rm4/checkpoint/input_embeddings_shard_000.pkl
copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_r

{'output_path': '/tmp/jobs/local/results'}

In [11]:
#For running model on custom input
"""
Q's:
- How much structure does data need before feeding into policy?(Dict? String?)
"""
from summarize_from_feedback import task_data
from summarize_from_feedback.policy import Policy

H = HParams

layout = H.model_spec.run_params.all_gpu_layout()

# Instantiate policy
policy = Policy(task_hparams=H.task, spec=H.model_spec, layout=layout)

#if H.orig_model_spec:
#    assert H.orig_model_spec.run_params.n_shards == H.model_spec.run_params.n_shards
#    orig_policy = Policy(task_hparams=H.task, spec=H.orig_model_spec, layout=layout)
#else:
#    orig_policy = None

encoder = policy.encoder
#response_encoder = ResponseEncoder(H.task.response, encoder)
#setup_logging_with_pacific_tz()

#act_dtype = torch.float16 if H.fp16_activations else torch.float32

#is_logging_rank = layout.is_logging_rank

#total_queries_per_replica = exact_div(H.num_queries, layout.n_replicas)
#num_runs = exact_div(total_queries_per_replica, H.queries_per_run_per_replica)

input_iter = task_data.get_iter_for_task(
    H.task,
    encoder=encoder,
    dataset_split=H.query_dataset_split,
    batch_size=H.queries_per_run_per_replica,
    layout=layout,
    seed=H.seed,
    all_fields=True,
    )
context = next(input_iter)

Constructing Transformer with the following model hparams:
model_H:
attn_dropout: 0.0
d_model: 2048
emb_dropout: 0.0
fp16_conv_weights: False
fp16_embedding_weights: False
heads: 16
include_pos_embeddings: True
init_scale: 0.7
key_bias: None
m_attn: 1
m_mlp: 4
n_ctx: 2048
n_layer: 24
n_shards: 1
res_scale: False
resid_dropout: 0.0
text_encoding: reversible_50000
use_blocksparse_attn: True
zero_out: False
Loaded model to cuda:0. CUDA memory allocated: 11.35 GB


In [88]:
#H.task.query.dataset #DEBUG
#Padding method for ensuring proper context size for policy
PADDING_TOKEN = encoder.encode(" ")
def pad(token_list, desired_shape=512):
    diff = len(token_list) - desired_shape
    if diff < 0:
        token_list = PADDING_TOKEN * abs(diff) + token_list
    return token_list[:512]

In [90]:
raw_context = encoder.decode(context["context"]["tokens"])
sample = policy.sample(context["context"]["tokens"])
sample_1 = policy.sample(torch.tensor([pad(encoder.encode(raw_context))]))

-2


In [94]:
#Testing out impact of extracted method of encoding + leaving out POST, SUBREDDIT and TL;DR strings in context. 
for sample in list(map(lambda sample: encoder.decode(sample["samples"]),[sample,sample_1])):
     print(sample)

 I had a difficulty in my lin algebra final a Friday night later and I aced it and now go drinking the night before every final.  What are/were your final pre-college routine?<|endoftext|>

In a new book,
 took intense math class freshman year and didn't get it, woke up the next morning hung over and went to take test, aced final, now go drinking every final night and it works.<|endoftext|>In response to a question on Facebook,


In [ ]:
print(torch.tensor([pad(encoder.encode(raw_context))]).size())
print(context["context"]["tokens"].size())

In [95]:
print(raw_context)

                                                                                                                                                                                                                                                                                                           SUBREDDIT: r/AskReddit

TITLE: I have a final for my summer class tomorrow and I thought I'd ask everyone:  What are/were you're pre- college final routines?

POST: Some story.  My freshman year of college first semester I took this extremely difficult linear algebra class.  I was between an F-D all semester but arrogant me would just say after every bad test "Ah I'll figure it out, I don't need to drop."  Well it came to the night before the final (a Friday and my first college final) and I still didn't fucking get it.  So I said fuck it and went out and got shit faced with my friends.  I woke up the next morning hung over as all hell and went to take my test.  Low and behold I aced the fuck

In [96]:
custom_context = """
SUBREDDIT: r/Math

TITLE: What's a theorem that made you furl your eyebrows and say "what?!" the first time you saw it?

POST: Title is self explanatory. Tell me some theorems/results that are unintuitive or unexpected! Perhaps a good example of this would be seeing the solution to the Bassel Problem for the first time. Bonus points if the theorem/result is either really beautiful or really ugly! Edit: It’s been pointed out to me that most people are unable to fuel their eyebrows, and that the appropriate word here is “furrow”. However, please still feel free to share if a theorem shocked you so much that you indeed managed to furl your eyebrows in disbelief!

TL;DR:
"""
print(custom_context)


SUBREDDIT: r/Math

TITLE: What's a theorem that made you furl your eyebrows and say "what?!" the first time you saw it?

POST: Title is self explanatory. Tell me some theorems/results that are unintuitive or unexpected! Perhaps a good example of this would be seeing the solution to the Bassel Problem for the first time. Bonus points if the theorem/result is either really beautiful or really ugly! Edit: It’s been pointed out to me that most people are unable to fuel their eyebrows, and that the appropriate word here is “furrow”. However, please still feel free to share if a theorem shocked you so much that you indeed managed to furl your eyebrows in disbelief!

TL;DR:



In [100]:
custom_sample = policy.sample(torch.tensor([pad(encoder.encode(custom_context))]))
print("Output: {}".format(encoder.decode(custom_sample["samples"])))

-346
Output:  concluding some math problems for the first time, saw a theorem that made you furrow your eyebrows and say "what?!" the first time you saw it. Share your own experiences with theorems and results!<|endoftext|>This is a post
